In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import pandas as pd
from os import path
import glob
import warnings
from scrapers.scrapepool import ScrapePool
from scrapers.wos import WosScraper
import linecache
import csv
from beakerx import *

# disable warnings, because pandas throws warnings for mixed type columns all the time..
warnings.filterwarnings('ignore')

%autoreload

In [3]:
# scraper settings
args = {
    'phantomjs_exec': 'C:\\phantomjs.exe',
    'urls': {
        #'author_search': "http://apps.webofknowledge.com/WOS_AuthorSearch.do?product=WOS&action=Search&search_mode=AuthorSearch&AU=",
        'author_search': "http://apps.webofknowledge.com/DaisyOneClickSearchNoHistory.do?search_mode=DaisyOneClickSearchNoHistory&parentProduct=WOS&parentQid=10036&product=WOS&author_name=",
        'save_to': 'data\\wos\\out_feb18_2\\'
    },
    'topic': "",
    'num_pubs': 5
}

In [ ]:
df = pd.read_csv("data/wos/2017_HCR_as_of_11_16_2017_fittopublish.csv", sep=",", quotechar="\"", encoding="ISO-8859-1")
searched = set(["_".join(".".join(path.basename(f).split(".")[:-1]).split("_")[:-1]) for f in glob.glob(path.join(args['urls']['save_to'], "*.*")) if path.getsize(f) > 0])
keywords = list(set(df['NAME'].tolist())-searched)
print(str(len(keywords)))

In [ ]:
s = ScrapePool(WosScraper, keywords, args, chunk_size=300)
results = s.run()

In [6]:
merged = dict()
for f in glob.glob(path.join(args['urls']['save_to'], "*.*")):
    if path.getsize(f) > 4:
        name= "_".join(".".join(path.basename(f).split(".")[:-1]).split("_")[:-1])
        merged[name] = linecache.getline(f, 2).replace("\"","").replace(";",",").rstrip()

In [ ]:
df = df.set_index('name')
df

In [ ]:
df2 = pd.DataFrame.from_dict(merged, orient='index')
df2.columns = ['email']
df2

In [ ]:
all_merged = pd.concat([df, df2], axis=1, join_axes=[df.index])
all_merged

In [8]:
def get_main_email(x):
    all_mails = [(e, e.split("@")[0]) for e in x["email"].split(",")]
    return ",".join([e for e, en in all_mails if x["lastname"].lower() in en.lower()])

In [ ]:
df2["name"] = df2.index
df2["lastname"] = df2.apply(lambda x: x["name"].split(",")[0].lower(), axis=1)
df2["main_email"] = df2.apply(get_main_email, axis=1)
df2

In [45]:
df2.to_csv(path.join(args['urls']['save_to'], 'merged.csv'), sep=";", index=True, quotechar='"', quoting=csv.QUOTE_ALL)

## sent

In [64]:
df_sent = pd.read_csv("data/wos/sent.csv", sep=",")
df_main_auth = pd.DataFrame([], columns=["email"])

In [ ]:
main = [e.split(",")[0].lower() for e in df2["main_email"].tolist()]
name_dict = {e.split(",")[0].lower(): n for e, n in zip(df2["main_email"].tolist(), df2["name"].tolist())}
sent = []#[e.lower() for e in df_sent["email"]]
new = set([e for e in main if e not in sent and not e==""])
pd.DataFrame({
    "email": list(new),
    "name": [name_dict[e] for e in list(new)]
})

In [ ]:
#co_auth = [e.split(",") for e in df2["email"].tolist()]
co_auth = [email.lower() for e_list in df2["email"].tolist() for email in e_list.split(",")]
main_auth = [email.lower() for e_list in df2["main_email"].tolist() for email in e_list.split(",")]
sent_main = sent + main_auth
new_co_auth = set([e for e in co_auth if e not in sent_main and not e==""])
df_new_co_auth = pd.DataFrame(list(new_co_auth), columns=["email"])
print(len(df_new_co_auth))
df_new_co_auth